In [1]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import dotenv_values
from pycaret.clustering import setup, create_model, assign_model, plot_model, save_model, load_model, predict_model

In [2]:
env = dotenv_values(".env")

openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

In [3]:
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
df.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


In [4]:
len(df)

229

In [5]:
s = setup(df, session_id=123)
s

,Description,Value
0,Session id,123
1,Original data shape,"(229, 5)"
2,Transformed data shape,"(229, 21)"
3,Categorical features,5
4,Rows with missing values,13.1%
5,Preprocess,True
6,Imputation type,simple
7,Numeric imputation,mean
8,Categorical imputation,mode
9,Maximum one-hot encoding,-1


In [6]:
kmeans = create_model('kmeans', num_clusters=8)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.1925,27.1789,1.8733,0,0,0


Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
df_with_clusters = assign_model(kmeans)
df_with_clusters

,age,edu_level,fav_animals,fav_place,gender,Cluster
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta,Cluster 7
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna,Cluster 7
2,45-54,Wyższe,Psy,W lesie,Mężczyzna,Cluster 3
3,35-44,Średnie,Koty,W górach,Mężczyzna,Cluster 4
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna,Cluster 0
...,...,...,...,...,...,...
224,35-44,Wyższe,Koty,Inne,Kobieta,Cluster 2
225,45-54,Wyższe,Inne,W lesie,Mężczyzna,Cluster 3
226,25-34,Wyższe,Psy,W górach,Mężczyzna,Cluster 1
227,35-44,Wyższe,Brak ulubionych,W górach,Mężczyzna,Cluster 2


In [8]:
df_with_clusters["Cluster"].value_counts()

Cluster
Cluster 3    38
Cluster 1    34
Cluster 0    30
Cluster 6    30
Cluster 5    29
Cluster 4    26
Cluster 2    23
Cluster 7    19
Name: count, dtype: int64

In [9]:
save_model(kmeans, 'welcome_survey_clustering_pipeline_v2', verbose=False)

(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=[], transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['age', 'edu_level', 'fav_animals',
                                              'fav_place', 'gender'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_encoding',
                  TransformerWrapper(include=['gender'],
                                     transfo...
                                                                mapping=[{'col': 'gender',
                                                                          'data_type': dtype('O'),
                                                                          'mapping': Kobieta      0
 Mężczyzna    1
 NaN         -1
 dtype: int64}]))),
                 ('onehot_encoding',
                  TransformerWrappe

In [10]:
cluster_descriptions = {}
for cluster_id in df_with_clusters['Cluster'].unique():
    cluster_df = df_with_clusters[df_with_clusters['Cluster'] == cluster_id]
    summary = ""
    for column in df_with_clusters:
        if column == 'Cluster':
            continue

        value_counts = cluster_df[column].value_counts()
        value_counts_str = ', '.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()])
        summary += f"{column} - {value_counts_str}\n"

    cluster_descriptions[cluster_id] = summary

In [11]:
prompt = "Użyliśmy algorytmu klastrowania."
for cluster_id, description in cluster_descriptions.items():
    prompt += f"\n\nKlaster {cluster_id}:\n{description}"

prompt += """
Wygeneruj najlepsze nazwy dla każdego z klasterów oraz ich opisy

Użyj formatu JSON. Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
}
"""
print(prompt)

Użyliśmy algorytmu klastrowania.

Klaster Cluster 7:
age - 35-44: 8, 45-54: 4, 25-34: 3, 18-24: 2, <18: 1, >=65: 1, 55-64: 0, unknown: 0
edu_level - Średnie: 18, Podstawowe: 1, Wyższe: 0
fav_animals - Psy: 13, Koty: 3, Brak ulubionych: 2, Inne: 1, Koty i Psy: 0
fav_place - Nad wodą: 12, W lesie: 2, Inne: 0, W górach: 0
gender - Mężczyzna: 13, Kobieta: 6


Klaster Cluster 3:
age - 45-54: 18, 35-44: 8, 25-34: 7, 55-64: 4, >=65: 1, 18-24: 0, <18: 0, unknown: 0
edu_level - Wyższe: 38, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 23, Brak ulubionych: 5, Inne: 5, Koty: 5, Koty i Psy: 0
fav_place - W lesie: 38, Inne: 0, Nad wodą: 0, W górach: 0
gender - Mężczyzna: 29, Kobieta: 9


Klaster Cluster 4:
age - 45-54: 9, 25-34: 7, 18-24: 6, 35-44: 4, 55-64: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Średnie: 26, Podstawowe: 0, Wyższe: 0
fav_animals - Koty: 9, Inne: 7, Psy: 7, Brak ulubionych: 3, Koty i Psy: 0
fav_place - W górach: 18, W lesie: 5, Inne: 3, Nad wodą: 0
gender - Mężczyzna: 24, Kobieta

In [12]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
)

In [13]:
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions = json.loads(result)

In [14]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [16]:
with open("welcome_survey_cluster_names_and_descriptions_v2.json", "r") as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Miłośnicy Wody z Wyższym Wykształceniem', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu nad wodą. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 1': {'name': 'Górscy Profesjonaliści', 'description': 'W tym klastrze znajdują się osoby z wyższym wykształceniem, które preferują spędzanie czasu w górach. Dominują osoby w wieku 45-54 lat, a większość z nich to mężczyźni. Ich ulubionymi zwierzętami są psy.'}, 'Cluster 2': {'name': 'Kociarze z Wyższym Wykształceniem', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu w górach. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są koty.'}, 'Cluster 3': {'name': 'Leśni Entuzjaści z Wyższym Wykształceniem', 'description': 'W tym klastrze znajdują się osoby z wyższym wykształceniem, które preferują spędzanie czasu w lesie. Dom

In [ ]:
{
  "diagram": {
    "wykresy": {
      "wykres_1": {
        "kolor": "żółty",
        "nazwa": "Inteligencja emocjonalna i wynik trenera",
        "elementy": {
          "Inteligencja emocjonalna": ["Komunikacja", "Zaangażowanie", "Autentyczność", "Zarządzanie konfliktami"],
          "Komunikacja": ["Kontrola klimatu emocjonalnego", "Przywództwo"],
          "Zaangażowanie": ["Przywództwo"],
          "Autentyczność": ["Przywództwo"],
          "Zarządzanie konfliktami": ["Przywództwo"],
          "Kontrola klimatu emocjonalnego": ["Zaufanie wzajemne", "Przywództwo"],
          "Przywództwo": ["Ekspertyza w grze", "Zaufanie wzajemne"],
          "Ekspertyza w grze": ["Zaufanie wzajemne"],
          "Wynik trenera": ["Kontrola klimatu emocjonalnego", "Przywództwo", "Ekspertyza w grze"]
        }
      },
      "wykres_2": {
        "kolor": "niebieski",
        "nazwa": "Równowaga afektywna i wynik gracza",
        "elementy": {
          "Równowaga afektywna": ["Umiejętności regulacji emocjonalnej", "Stan emocjonalny", "Motywacja"],
          "Umiejętności regulacji emocjonalnej": ["Pewność siebie", "Stan emocjonalny"],
          "Stan emocjonalny": ["Pewność siebie", "Motywacja"],
          "Motywacja": ["Wynik gracza"],
          "Pewność siebie": ["Wynik gracza"],
          "Wynik gracza": ["Umiejętności społeczne", "Umiejętności poznawcze", "Umiejętności motoryczne", "Umiejętności fizyczne"],
          "Umiejętności społeczne": ["Wynik grupy"],
          "Umiejętności poznawcze": ["Wynik grupy"],
          "Umiejętności motoryczne": ["Wynik grupy"],
          "Umiejętności fizyczne": ["Wynik grupy"]
        }
      },
      "wykres_3": {
        "kolor": "czerwony",
        "nazwa": "Wynik grupy i strategia",
        "elementy": {
          "Wynik grupy": ["Inteligencja zbiorowa", "Struktura grupy", "Spójność grupy", "Skuteczność grupowa", "Strategia gry"],
          "Inteligencja zbiorowa": ["Tożsamość grupy", "Normy i wartości", "Komunikacja", "Role i statusy"],
          "Struktura grupy": ["Tożsamość grupy", "Normy i wartości", "Komunikacja", "Role i statusy"],
          "Spójność grupy": ["Tożsamość grupy", "Normy i wartości", "Komunikacja", "Role i statusy"],
          "Skuteczność grupowa": ["Tożsamość grupy", "Normy i wartości", "Komunikacja", "Role i statusy"],
          "Strategia gry": ["Tożsamość grupy", "Normy i wartości", "Komunikacja", "Role i statusy"]
        }
      },
      "wykres_4": {
        "kolor": "zielony",
        "nazwa": "Wynik sportowy struktury",
        "elementy": {
          "Wynik sportowy struktury": ["Organizacja funkcjonalna"],
          "Organizacja funkcjonalna": ["Wartości", "Środki", "Kultura", "Wspólna wizja"]
        }
      }
    },
    "relacje_globalne": {
      "Zaufanie wzajemne": ["Wynik gracza", "Wynik grupy"],
      "Wynik gracza": ["Wynik grupy"],
      "Wynik grupy": ["Wynik sportowy struktury"]
    },
    "środowiska": {
      "Środowisko medialne": {},
      "Środowisko społeczno-ekonomiczne": {},
      "Środowisko zawodowe": {},
      "Środowisko afektywne": {}
    }
  }
}
